In [37]:
from itertools import count

import pandas as pd
import numpy as np
import os

In [38]:
# Load the dataset
file_path = os.path.join("..", "data", "chargeback.csv")
df = pd.read_csv(file_path)
# Delete Unnamed column
df.drop(df.columns[0], axis=1, inplace = True)
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Data Dictionary

index - Unique Identifier for each row

trans_date_trans_time - Transaction DateTime

cc_num - Credit Card Number of Customer

merchant - Merchant Name

category - Category of Merchant

amt - Amount of Transaction

first - First Name of Credit Card Holder

last - Last Name of Credit Card Holder

gender - Gender of Credit Card Holder

street - Street Address of Credit Card Holder

city - City of Credit Card Holder

state - State of Credit Card Holder

zip - Zip of Credit Card Holder

lat - Latitude Location of Credit Card Holder

long - Longitude Location of Credit Card Holder

city_pop - Credit Card Holder's City Population

job - Job of Credit Card Holder

dob - Date of Birth of Credit Card Holder

trans_num - Transaction Number

unix_time - UNIX Time of transaction

merch_lat - Latitude Location of Merchant

merch_long - Longitude Location of Merchant

is_fraud - Fraud Flag <--- Target Class

In [39]:
# Initialize columns
df['is_chargeback'] = 0
df['visa_reason_code'] = None
print(len(df))
df.head()

1296675


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,is_chargeback,visa_reason_code
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,0,None
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,0,None
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,0,None
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,0,None
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,0,None


In [40]:
# If the transaction is fraud, then it'll become a chargeback with the visa reason code "10.4 - Card Absent"
df_fraud = df['is_fraud'] == 1
df.loc[df_fraud, 'is_chargeback'] = 1
df.loc[df_fraud, 'visa_reason_code'] = '10.4 - Card Absent'

df_check_fraud = df[df['is_fraud'] == 1].copy()
print(len(df_check_fraud))
df_check_fraud.head()

7506


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,is_chargeback,visa_reason_code
2449,2019-01-02 01:06:37,4613314721966,fraud_Rutherford-Mertz,grocery_pos,281.06,Jason,Murphy,M,542 Steve Curve Suite 011,Collettsville,...,885,Soil scientist,1988-09-15,e8a81877ae9a0a7f883e15cb39dc4022,1325466397,36.430124,-81.179483,1,1,10.4 - Card Absent
2472,2019-01-02 01:47:29,340187018810220,"fraud_Jenkins, Hauck and Friesen",gas_transport,11.52,Misty,Hart,F,27954 Hall Mill Suite 575,San Antonio,...,1595797,Horticultural consultant,1960-10-28,bc7d41c41103877b03232f03f1f8d3f5,1325468849,29.819364,-99.142791,1,1,10.4 - Card Absent
2523,2019-01-02 03:05:23,340187018810220,fraud_Goodwin-Nitzsche,grocery_pos,276.31,Misty,Hart,F,27954 Hall Mill Suite 575,San Antonio,...,1595797,Horticultural consultant,1960-10-28,b98f12f4168391b2203238813df5aa8c,1325473523,29.273085,-98.836360,1,1,10.4 - Card Absent
2546,2019-01-02 03:38:03,4613314721966,fraud_Erdman-Kertzmann,gas_transport,7.03,Jason,Murphy,M,542 Steve Curve Suite 011,Collettsville,...,885,Soil scientist,1988-09-15,397894a5c4c02e3c61c784001f0f14e4,1325475483,35.909292,-82.091010,1,1,10.4 - Card Absent
2553,2019-01-02 03:55:47,340187018810220,fraud_Koepp-Parker,grocery_pos,275.73,Misty,Hart,F,27954 Hall Mill Suite 575,San Antonio,...,1595797,Horticultural consultant,1960-10-28,7863235a750d73a244c07f1fb7f0185a,1325476547,29.786426,-98.683410,1,1,10.4 - Card Absent


In [41]:
df['category'].unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

### Math Logic

The current dataset has 1,296,675 rows and 7,506 rows are fraud.

For the purpose of this project, we want the company to be in the danger zone (0.8% chargeback ratio).

1,296,675 * 0.008 = 10,373 chargebacks. Since we already have 7,506 chargebacks from fraud, that means 10,373 - 7,506 = 2867 chargebacks needed to add to reach the 0.8% chargeback ratio.

We're going to target the online purchases to be the bad merchants which is 203,520 rows.

(2867 / 203,520) * 100 = 1.41% will be our random sample.


In [42]:
# Turning some online purchases into chargebacks with reason code "13.1 - Merchandise Not Received"
_net = ['misc_net', 'grocery_net', 'shopping_net',]
df_friendly_fraud = (df['category'].isin(_net)) & (df['is_fraud'] == 0)
df_friendly_fraud


0           True
1          False
2          False
3          False
4          False
           ...  
1296670    False
1296671    False
1296672    False
1296673    False
1296674    False
Length: 1296675, dtype: bool

In [43]:
# Sample 1.41% to hit the 0.8% Danger Zone
friendly_fraud_sample = df[df_friendly_fraud].sample(frac=0.0141, random_state=42).index

# Add the Friendly Fraud to these random samples
df.loc[friendly_fraud_sample, 'is_chargeback'] = 1
df.loc[friendly_fraud_sample, 'visa_reason_code'] = '13.1 - Merchandise Not Received'

df[df['is_chargeback'] == 1].head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,is_chargeback,visa_reason_code
354,2019-01-01 04:37:51,30487648872433,"fraud_Rohan, White and Aufderhar",misc_net,2.73,Stephanie,Crane,F,144 Martinez Curve,Central,...,350,Counsellor,1955-01-05,9445dc9bc93c2020f5364655a9fcd5f5,1325392671,37.997357,-86.714460,0,1,13.1 - Merchandise Not Received
367,2019-01-01 04:50:40,4658490815480264,"fraud_Streich, Dietrich and Barton",shopping_net,9.34,Tara,Richards,F,4879 Cristina Station,Keisterville,...,184,Systems developer,1945-11-04,012c301621c84a9ea488dcce8b338f47,1325393440,39.646451,-80.502965,0,1,13.1 - Merchandise Not Received
473,2019-01-01 06:15:36,6011860238257910,fraud_Medhurst PLC,shopping_net,10.09,Lisa,Mendez,F,44259 Beth Station Suite 215,Lahoma,...,1078,"Programme researcher, broadcasting/film/video",1952-07-06,cbf663d4480e0447c14c77d383fb5532,1325398536,36.120627,-97.940436,0,1,13.1 - Merchandise Not Received
885,2019-01-01 11:10:30,4500002361389,fraud_Ortiz Group,grocery_net,54.15,Joseph,Nixon,M,86882 Desiree Camp Suite 047,Tyaskin,...,470,Music tutor,1965-04-07,aedb8df3d5bc7fe6342c2dc8f102360d,1325416230,38.397353,-75.303196,0,1,13.1 - Merchandise Not Received
1224,2019-01-01 14:31:02,4189814094741,fraud_Cormier LLC,shopping_net,3.73,Jason,Farmer,M,33165 Larry Walks Suite 960,Ragland,...,4159,Educational psychologist,1998-02-03,f8bc717f13fd65dcf9c82513569e39ee,1325428262,34.353078,-85.643205,0,1,13.1 - Merchandise Not Received


In [44]:
# Confirming the company chargeback ratio
company_chargeback_ratio = (np.sum(df['is_chargeback'] == 1) / len(df)) * 100
company_chargeback_ratio


np.float64(0.8002005128501746)

In [45]:
# Checking the merchant's chargeback ratios
merchant_chargeback_ratio = df.groupby(['merchant', 'category']).agg(
    total_transactions=('is_chargeback', 'count'),
    total_chargebacks=('is_chargeback', 'sum')
)

merchant_chargeback_ratio['chargeback_ratio'] = (merchant_chargeback_ratio['total_chargebacks'] / merchant_chargeback_ratio['total_transactions']) * 100

merchant_chargeback_ratio.sort_values(by='chargeback_ratio', ascending=False, inplace=True)

merchant_chargeback_ratio

,,total_transactions,total_chargebacks,chargeback_ratio
merchant,category,,,
fraud_Miller-Harris,misc_net,1241,52,4.190169
fraud_Terry-Huel,shopping_net,1996,80,4.008016
fraud_Kozey-Boehm,shopping_net,1866,71,3.804930
fraud_Brown PLC,misc_net,1176,44,3.741497
fraud_Kuhn LLC,misc_net,1222,45,3.682488
...,...,...,...,...
fraud_Reichert-Weissnat,travel,753,0,0.000000
fraud_Gulgowski LLC,home,2513,0,0.000000
"fraud_Larson, Quitzon and Spencer",travel,821,0,0.000000


In [46]:
# Export the data to csv in the data folder
output_path = os.path.join("..", "data", "chargeback_update.csv")
df.to_csv(output_path, index=False)
print(f"File successfully saved to: {output_path}")

File successfully saved to: ..\data\chargeback_update.csv
